# 5 Missing Values

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('train.csv')

### Missing Data

Important questions when thinking about missing data:
- How prevalent is the missing data?
- Is missing data random or does it have a pattern?

The answer to these questions is important for practical reasons because missing data can imply a reduction of the sample size. This can prevent us from proceeding with the analysis. Moreover, from a substantive perspective, we need to ensure that the missing data process is not biased and hidding an inconvenient truth.

In [2]:
df.Alley.unique()

array([nan, 'Grvl', 'Pave'], dtype=object)

In [3]:
#missing data
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
PoolQC,1453,207.571429
MiscFeature,1406,26.037037
Alley,1369,15.043956
Fence,1179,4.195730
FireplaceQu,690,0.896104
LotFrontage,259,0.215654
GarageCond,81,0.058738
GarageType,81,0.058738
GarageYrBlt,81,0.058738
GarageFinish,81,0.058738


#### Let's analyse this to understand how to handle the missing data. 

In some cases, missing data indicates that a value is simply not applicable. Notice for instance that the variables whose names start with 'Garage' all have the same number of missing values. This is likely because 81 of the houses do not have a garage. To use a variable like GarageType, we could replace the missing values with a new category 'No Garage'. The same logic applies to variables whose names start with 'Bsmt' which is short for Basement.

Alley has a high percentage of missing data. The code book tells us the values are Paved, Gravel or NA (not available). It appears NA was incorrectly interpreted by Python as a missing value, so we can safely replace the missing values with 'NA' to restore those. The same applies to PoolQC and Fence; for Fence NA means 'No Fence'.

The variable LotFrontage means the number of feet that the lot is connected to the main road. For LotFrontage there are 18% missing values, possibly because 18% of the houses does not have a lot. If we were to replace those values with 0, we might be able to use the variable in the prediction of SalePrice. On the other hand, LotFrontage may not matter that much, so we may be ok to just to just delete it. In fact, in this tutorial we will simply remove all variables with > 15% mising values. However, if one of these variables is interesting we should not dismiss them too easily. 

Regarding 'MasVnrArea' and 'MasVnrType', we can consider that these variables are not essential. Furthermore, they have a strong correlation with 'YearBuilt' and 'OverallQual' which are already considered. Thus, we are not likely to lose information if we delete 'MasVnrArea' and 'MasVnrType'.

Finally, we have one missing observation in 'Electrical'. Since it is just one observation, we'll delete this observation and keep the variable. Yo have to be careful though, not to remove too many samples or to bias your dataset by removing samples.

In summary, to handle missing data, we'll delete all the variables with missing data, except the variable 'Electrical'. In 'Electrical' we'll just delete the observation with missing data

In [4]:
#dealing with missing data
df = df.drop((missing_data[missing_data['Total'] > 1]).index,1)
df = df.drop(df.loc[df['Electrical'].isnull()].index)
df.isnull().sum().max() #just checking that there's no missing data missing...

0

#### Assignment: 

inspect what df.drop(df.loc[df['Electrical'].isnull()].index) does by inspecting 

In [5]:
df.drop(df.loc[df['Electrical'].isnull()].index)

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000
5,6,50,RL,14115,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,320,0,0,700,10,2009,WD,Normal,143000
6,7,20,RL,10084,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,8,2007,WD,Normal,307000
7,8,60,RL,10382,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,228,0,0,0,350,11,2009,WD,Normal,200000
8,9,50,RM,6120,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,205,0,0,0,0,4,2008,WD,Abnorml,129900
9,10,190,RL,7420,Pave,Reg,Lvl,AllPub,Corner,Gtl,...,0,0,0,0,0,1,2008,WD,Normal,118000


df.drop(df.loc[df['Electrical'].isnull()].index)

- Uit het dataframe wordt iets 'gedropt' (verwijderd) 
- Hetgene dat verwijderd (df.loc[df['Electrical'].isnull()].index) zijn alle records waar in colom 'Electrical' geen waarde staat